In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import FunctionLib
#%matplotlib widget
basePath ='./sims.TNG/TNG50-1/output'
from tqdm import tqdm
import gc
import os

In [2]:
snapshot_init=67
snapshot_end=99
lbsg_index_list=[117251,300903,324123,329508,382215,400973,419618,457431,460722,490814]
m_dm=4.5E-5  #E10M_Sum
h=0.6774  #km/Mpc
Critial_Density=1.27E-8  #E10M_Sum/kpc^3
Critial_Density_h=Critial_Density/h**2  #E10M_Sum/(kpc/h)^3
lbsg_index_array=np.array(lbsg_index_list)

In [3]:
gas_fields=['Coordinates','Masses','Velocities']
dm_fields=['Coordinates','Velocities']
stars_fields=['Coordinates','Masses','Velocities']
bh_fields=['Coordinates','Masses','Velocities']
tree_fields=['SubfindID','SnapNum','Mass']
subhalo_fields=['SubhaloPos','SubhaloVel','SubhaloSpin','SubhaloCM','SubhaloMass']

In [4]:
def Cal_M_200_From_R_200(R_200,Comoving_Critial_Density):
    
    M_200=200*Comoving_Critial_Density*4/3*3.14*R_200**3
    return M_200

def R_200_Calculation(snapshot_num,subhalos_data,subhalo_id,Comoving_Critial_Density):
    
    subhalo_comoving_position=subhalos_data['SubhaloPos'][subhalo_id]
    subhalo_comoving_velocity=subhalos_data['SubhaloVel'][subhalo_id]
    subhalo_spin=subhalos_data['SubhaloSpin'][subhalo_id]
    subhalo_mass=subhalos_data['SubhaloMass'][subhalo_id]
    
    gas_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,0,fields=gas_fields)
    dm_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,1,fields=dm_fields)
    stars_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,4,fields=stars_fields)
    bh_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,5,fields=bh_fields)
    
    
    precision = 0.1
    max_iter = 100
    left, right = 0, 1000
    
    for i in range(max_iter):

        mid = (left + right) / 2
        
        mass_in=total_mass_in_radius(gas_data_tensor,dm_data_tensor,stars_data_tensor,bh_data_tensor,subhalo_comoving_position,mid)
        
        density=3*mass_in/(4*3.14*mid**3)
        
        if density > 200*Comoving_Critial_Density:
            left = mid

        else:
            right = mid
        

        if abs(right - left) < precision or i == max_iter - 1:
            R_200 = (left + right) / 2
            break
    
    M_200=Cal_M_200_From_R_200(R_200,Comoving_Critial_Density)
    
    return R_200,M_200

def baryon_mass_in_radius(Center,Radius,Coordinates,Masses):
    dist_to_com = np.sqrt(np.sum((Coordinates - Center)**2, axis=1))
    within_radius_idx = np.where(dist_to_com <= Radius)[0]
    total_mass_within_radius = np.sum(Masses[within_radius_idx])
    return total_mass_within_radius

def dm_mass_in_radius(Center,Radius,Coordinates):
    dist_to_com = np.sqrt(np.sum((Coordinates - Center)**2, axis=1))
    within_radius_idx = np.where(dist_to_com <= Radius)[0]
    total_mass_within_radius = m_dm*within_radius_idx.shape[0]*h
    return total_mass_within_radius

def total_mass_in_radius(gas_data_tensor,dm_data_tensor,stars_data_tensor,bh_data_tensor,Center,Radius):
    
    gas_mass_in=baryon_mass_in_radius(Center,Radius,gas_data_tensor['Coordinates'],gas_data_tensor['Masses'])
    dm_mass_in=dm_mass_in_radius(Center,Radius,dm_data_tensor['Coordinates'])
    stars_mass_in=baryon_mass_in_radius(Center,Radius,stars_data_tensor['Coordinates'],stars_data_tensor['Masses'])
    bh_mass_in=baryon_mass_in_radius(Center,Radius,bh_data_tensor['Coordinates'],bh_data_tensor['Masses'])
    
    total_mass_in=gas_mass_in+dm_mass_in+stars_mass_in+bh_mass_in
    
    return total_mass_in

def Binary_DataIter(Data_Set1,Data_Set2):
    for i in range(0,min(len(Data_Set1),len(Data_Set2))):
        yield Data_Set1[i],Data_Set2[i]
        
def Velocity_Virial_Calculation(R_200,M_200):
    Unit=4.3E4
    V=np.sqrt(R_200/M_200*Unit)
    return V
    
def cal_id_at_Z_from_99(Subhalo_Id_0,Z):#Z is the snapshot
    subfindid_snapnum_tree=il.sublink.loadTree(basePath, 99, Subhalo_Id_0, fields=tree_fields, onlyMPB=True)
    subfind_id=subfindid_snapnum_tree['SubfindID']
    snapshot_num=subfindid_snapnum_tree['SnapNum']
    index=np.where(snapshot_num==Z)[0]
    return subfind_id[index][0]

def Cal_Total_Spin(spin_array):
    len=0.0
    for axis_value in spin_array:
        len=len+axis_value**2
        
    return np.sqrt(len)

In [11]:
snapshot_end=70
snapshot_init=99
spin_parameter_matrix=np.zeros((abs(snapshot_end-snapshot_init+1),len(lbsg_index_list)))
for current_snapshot in tqdm(range(snapshot_init,snapshot_end-1,-1)):

    header=il.groupcat.loadHeader(basePath, current_snapshot)
    scale_factor=header['Time']
    comoving_critial_density=Critial_Density_h*(scale_factor**2)
    
    subhalos_data=il.groupcat.loadSubhalos(basePath,current_snapshot,fields=subhalo_fields)
    
    for subhalo_id_0 in lbsg_index_array:
        lbsg_index=np.where(lbsg_index_array==subhalo_id_0)[0][0]
        subhalo_id=cal_id_at_Z_from_99(subhalo_id_0,current_snapshot)

        R_200,M_200=R_200_Calculation(current_snapshot,subhalos_data,subhalo_id,comoving_critial_density)
        V_200=Velocity_Virial_Calculation(R_200,M_200)
        
        Spin_Array=subhalos_data['SubhaloSpin'][subhalo_id]
        Total_Spin=Cal_Total_Spin(Spin_Array)
        
        spin_parameter=Total_Spin*scale_factor/(V_200*R_200)
        spin_parameter_matrix[abs(snapshot_init-current_snapshot)][lbsg_index]=spin_parameter
        
    gc.collect()

 93%|█████████▎| 28/30 [1:36:41<06:54, 207.20s/it]


IndexError: index 28 is out of bounds for axis 0 with size 28

In [12]:
spin_parameter_matrix

array([[ 0.1951532 ,  0.30266728,  0.19418416,  0.19268461,  0.14101168,
         0.13458659,  0.12065335,  0.06890232,  0.09056041,  0.08255872],
       [ 0.20908652,  0.29875545,  0.18864785,  0.17446712,  0.13798   ,
         0.1281866 ,  0.11766484,  0.06815747,  0.08723384,  0.08064944],
       [ 0.19830108,  0.28945212,  0.17809205,  0.16934388,  0.13337887,
         0.12308849,  0.11012375,  0.06447966,  0.01742224,  0.07786846],
       [ 0.20481406,  0.28380561,  0.17222998,  0.18035348,  0.12751607,
         0.12086147,  0.10647643,  0.06312393,  0.01681751,  0.07582787],
       [ 0.21516041,  0.27736132,  0.15947957,  0.15960686,  0.12117844,
         0.11703192,  0.1024594 ,  0.06136772,  0.08493156,  0.07428272],
       [ 0.21431687,  0.27255977,  0.15873282,  0.1580072 ,  0.11796993,
         0.11352438,  0.09234806,  0.06050346,  0.08124237,  0.07330201],
       [ 0.2135746 ,  0.26206497,  0.14433657,  0.15364229,  0.11387717,
         0.11102651,  0.09529911,  0.05632502

In [26]:
id_values=np.array(lbsg_index_array)
t_values=np.array(range(snapshot_init,snapshot_end+1,-1))

In [27]:
id_values.shape

(10,)

In [21]:
t_values.shape

(28,)

In [19]:
spin_parameter_matrix.shape

(28, 10)

In [32]:
num_snapshots=len(t_values)
num_galaxies=len(id_values)

for x_index,x in enumerate(id_values):
    data=[spin_parameter_matrix[t_index][x_index] for t_index in range(num_snapshots)]
    
    plt.plot(t_values,data)
    plt.xlabel('Snapshot')
    plt.ylabel('Spin Parameter Evolution')
    plt.title(f'Spin Parameter of Galaxy {x}')
    
    plt.savefig(f'{x}.png')
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [29]:
np.save('./matrix.npy',spin_parameter_matrix)